In [1]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None


C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


我们在这里调用系统提供的Mnist数据函数为我们读入数据，如果没有下载的话则进行下载。

<font color=#ff0000>**这里将data_dir改为适合你的运行环境的目录**</font>

In [2]:
# Import data
data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

Extracting /tmp/tensorflow/mnist/input_data\train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data\t10k-labels-idx1-ubyte.gz


一个非常非常简陋的模型

In [3]:
# Create the model
tf.reset_default_graph()
x = tf.placeholder(tf.float32,[None,784])
learning_rate = tf.placeholder(tf.float32)

In [4]:
with tf.name_scope('reshape'):
  x_image = tf.reshape(x, [-1, 28, 28, 1])

# First convolutional layer - maps one grayscale image to 32 feature maps.
with tf.name_scope('conv1'):
  h_conv1 = tf.layers.conv2d(x_image, 32, [5,5],
                             padding='SAME',
                             activation=tf.nn.relu)

# Pooling layer - downsamples by 2X.
with tf.name_scope('pool1'):
  h_pool1 = tf.layers.max_pooling2d(h_conv1, pool_size=[2,2],
                        strides=[2, 2], padding='VALID')

# Second convolutional layer -- maps 32 feature maps to 64.
with tf.name_scope('conv2'):
  h_conv2 = tf.layers.conv2d(h_pool1, 64, [5,5],
                             padding='SAME',
                             activation=tf.nn.relu)

# Second pooling layer.
with tf.name_scope('pool2'):
  h_pool2 = tf.layers.max_pooling2d(h_conv2, pool_size=[2,2],
                        strides=[2, 2], padding='VALID')

# Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
# is down to 7x7x64 feature maps -- maps this to 1024 features.
with tf.name_scope('fc1'):
  h_pool2_flat = tf.layers.flatten(h_pool2)
  h_fc1 = tf.layers.dense(h_pool2_flat, 1024, activation=tf.nn.relu)

# Dropout - controls the complexity of the model, prevents co-adaptation of
# features.
with tf.name_scope('dropout'):
  keep_prob = tf.placeholder(tf.float32)
  h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Map the 1024 features to 10 classes, one for each digit
with tf.name_scope('fc2'):
  y = tf.layers.dense(h_fc1_drop, 10, activation=None)

定义我们的ground truth 占位符

In [5]:
# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

接下来我们计算交叉熵，注意这里不要使用注释中的手动计算方式，而是使用系统函数。
另一个注意点就是，softmax_cross_entropy_with_logits的logits参数是**未经激活的wx+b**

In [6]:
# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

l2_loss = tf.add_n( [tf.nn.l2_loss(w) for w in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)] )
total_loss = cross_entropy + 7e-5*l2_loss

生成一个训练step

In [7]:
train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

在这里我们仍然调用系统提供的读取数据，为我们取得一个batch。
然后我们运行3k个step(5 epochs)，对权重进行优化。

In [9]:
# Train
for step in range(3000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  lr = 0.001
  _, loss, l2_loss_value, total_loss_value = sess.run(
               [train_step, cross_entropy, l2_loss, total_loss], 
               feed_dict={x: batch_xs, y_: batch_ys, learning_rate:lr, keep_prob:0.5})
  
  if (step+1) % 100 == 0:
    print('step %d, entropy loss: %f, l2_loss: %f, total loss: %f' % 
            (step+1, loss, l2_loss_value, total_loss_value))
    # Test trained model
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5}))
  if (step+1) % 1000 == 0:
    print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels, keep_prob:0.5}))

step 100, entropy loss: 0.011288, l2_loss: 530.964050, total loss: 0.048456
1.0
step 200, entropy loss: 0.006289, l2_loss: 555.792114, total loss: 0.045195
1.0
step 300, entropy loss: 0.036923, l2_loss: 556.357666, total loss: 0.075868
0.97
step 400, entropy loss: 0.006352, l2_loss: 543.560486, total loss: 0.044402
1.0
step 500, entropy loss: 0.020167, l2_loss: 534.489258, total loss: 0.057581
0.99
step 600, entropy loss: 0.002813, l2_loss: 527.358704, total loss: 0.039728
1.0
step 700, entropy loss: 0.003738, l2_loss: 535.935242, total loss: 0.041254
1.0
step 800, entropy loss: 0.009652, l2_loss: 541.623352, total loss: 0.047566
1.0
step 900, entropy loss: 0.009358, l2_loss: 537.398376, total loss: 0.046976
1.0
step 1000, entropy loss: 0.027330, l2_loss: 534.167053, total loss: 0.064722
1.0
0.9916
step 1100, entropy loss: 0.004817, l2_loss: 553.718201, total loss: 0.043577
1.0
step 1200, entropy loss: 0.012984, l2_loss: 558.760254, total loss: 0.052097
1.0
step 1300, entropy loss: 0.0

验证我们模型在测试数据上的准确率

毫无疑问，这个模型是一个非常简陋，性能也不理想的模型。目前只能达到92%左右的准确率。
接下来，希望大家利用现有的知识，将这个模型优化至98%以上的准确率。
Hint：
- 卷积
- 池化
- 激活函数
- 正则化
- 初始化
- 摸索一下各个超参数
  - 卷积kernel size
  - 卷积kernel 数量
  - 学习率
  - 正则化惩罚因子
  - 最好每隔几个step就对loss、accuracy等等进行一次输出，这样才能有根据地进行调整